# Delta hedging of a spread options

Consider a market model with two assets, a Bank account and a 2 Stocks with risk neutral dynamics:

$$
dB(t) = rB(t) dt, \hspace{10mm} B(0) = 1 \\
dS_1(t) = r S_1(t) dt + \sigma_1 S_1(t) dW_1(t), \hspace{10mm} S_1(0) = s_1 > 0 \\
dS_2(t) = r S_2(t) dt + \sigma_2 S_2(t) dW_2(t), \hspace{10mm} S_2(0) = s_2 > 0
$$

where $\sigma_1, \sigma_2 > 0$ and where $E[dW_1(t) dW_2(t)] = \rho dt$ is the correlation between the two Brownian motions.

Consider a spread option with that at time $T$ pays $X = (S_2(T) - S_1(T))^+$.

Chope the time interval between now (time 0) and expiry of the call-option (time $T$) into $N$ pieces; denote the discretization points $t_i$.  

## Margrabe formula for princing the spread option

In the case where the strike price of a spread option is 0 ($K = 0$), there is a closed-form for pricing a spread option the Margrabe formula:

$$
p = x_2 N(d_1) - x_1 N(d_2)
$$

where 

$$
d_{1, 2} = \frac{\ln(x_2 / x_1)}{\sigma \sqrt{T}} \pm \frac{1}{2} \sigma \sqrt{T}
$$

and 

$$
x_1 = S_1(0), x_2 = S_2(0), \hspace{10mm} \sigma^2 = \sigma_1^2 + \sigma_2^2 - 2 \rho \sigma_1 \sigma_2
$$

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..\..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Discrete delta hedging experiment 

The hedging strategy is simple: just calcualte $\Delta_1$ and $\Delta_2$ at each time $t_i$ and then buy $\Delta_1$ units of stock 1 and $\Delta_2$ units of stock 2.

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import vmap

from jaxfin.models.gbm import MultiGeometricBrownianMotion

from src.spread import margrabe_deltas, margrabe, margrabe_cross_gamma, margrabe_gammas
import scienceplots
plt.style.use(['science','no-latex'])

SEED = 42
np.random.seed(SEED)

Parameters:

$S^1_0$ = $S^2_0$ = 100

$r = 0$

$\sigma_1 = \sigma_2 = 0.2$

$\rho = -0.5$

In [3]:
vmap_deltas = vmap(margrabe_deltas, in_axes=(0, 0, None, None, None, None))

In [4]:
def get_transaction_costs(ticksize, ddelta_1, ddelta_2):
    return ticksize * (jnp.abs(ddelta_1) + 0.01 * ddelta_1**2) \
        + ticksize * (jnp.abs(ddelta_2) + 0.01 * ddelta_2**2)

In [5]:
def hedge_experiment_loop(stock_paths, expiration, vols, correlation, N, ticksize):
    dt = expiration / N
    deltas = jnp.asarray([vmap_deltas(stock_paths[i, :, 0], stock_paths[i, :, 1], expiration - i * dt, vols[0], vols[1], correlation[0, 1]) for i in range(N)])
    spread_value = jnp.asarray([margrabe(stock_paths[i, :, 0], stock_paths[i, :, 1], expiration - i * dt, vols[0], vols[1], correlation[0, 1]) for i in range(N)])
    b = [spread_value[0, :] - deltas[0, :, 0] * stock_paths[0, :, 0] - deltas[0, :, 1] * stock_paths[0, :, 1] - get_transaction_costs(ticksize, deltas[0, :, 0], deltas[0, :, 1])]
    vs = [spread_value[0, :]]

    for i in range(1, N):
        ddelta_1 = deltas[i, :, 0] - deltas[i - 1, :, 0]
        ddelta_2 = deltas[i, :, 1] - deltas[i - 1, :, 1]
        transaction_cost = get_transaction_costs(ticksize, ddelta_1, ddelta_2)
        if i == N - 1:
            transaction_cost += get_transaction_costs(ticksize, deltas[i, :, 0], deltas[i, :, 1])

        vs.append(deltas[i - 1, :, 0] * stock_paths[i, :, 0] + deltas[i - 1, :, 1] * stock_paths[i, :, 1] + b[i - 1])
        b.append(vs[i] - deltas[i, :, 0] * stock_paths[i, :, 0] - deltas[i, :, 1] * stock_paths[i, :, 1] - transaction_cost)

    hedge_error = jnp.asarray(vs)[-1] - jnp.maximum(stock_paths[-1:, :, 1] - stock_paths[-1:, :, 0], 0)

    return jnp.mean(hedge_error), jnp.std(hedge_error) / jnp.asarray(spread_value)[0, 0]

## Hedging experiment without transaction costs

In [6]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [7]:
results = {
    'S1_0': [],
    'S2_0': [],
    'Volatility_1': [],
    'Volatility_2': [],
    'Maturity': [],
    'Rebalancing_freq': [],
    'Correlation': [],
    'E_PL_T': [],
    'Std_PL_T': []
}

In [8]:
def update_results(stock_paths, expiration, vols, correlation, N, ticksize=0.00):
    mean_hedge_error, std_hedge_error = hedge_experiment_loop(stock_paths, expiration, vols, correlation, N, ticksize)

    results['S1_0'].append(stock_paths[0, 0, 0])
    results['S2_0'].append(stock_paths[0, 0, 1])
    results['Volatility_1'].append(vols[0])
    results['Volatility_2'].append(vols[1])
    results['Maturity'].append(expiration)
    results['Rebalancing_freq'].append(N)
    results['Correlation'].append(correlation[0, 1])
    results['E_PL_T'].append(mean_hedge_error)
    results['Std_PL_T'].append(std_hedge_error)

    print(f'The mean hedge error is: {mean_hedge_error}')
    print(f'The normalized std of the hedge error is: {std_hedge_error}')

    return results

In [9]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.3005712032318115
The normalized std of the hedge error is: 0.0524766780436039


### Different starting stock prices

$$
S_1(0) = 100 \quad S_2(0) = 110
$$

In [10]:
s1_0 = 100
s2_0 = 110

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [11]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.330804705619812
The normalized std of the hedge error is: 0.037693582475185394


$$
S_1(0) = 100 \quad S_2(0) = 120
$$

In [12]:
s1_0 = 100
s2_0 = 120

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [13]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.2662851810455322
The normalized std of the hedge error is: 0.02767394483089447


$$
S_1(0) = 100 \quad S_2(0) = 80
$$

In [14]:
s1_0 = 100
s2_0 = 80

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [15]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.2408786565065384
The normalized std of the hedge error is: 0.12205778807401657


$$
S_1(0) = 100 \quad S_2(0) = 90
$$

In [16]:
s1_0 = 100
s2_0 = 90

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [17]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.2703498601913452
The normalized std of the hedge error is: 0.08254069089889526


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.3
$$

In [18]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.3, 0.3
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [19]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.4979100823402405
The normalized std of the hedge error is: 0.0523429736495018


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1
$$

In [20]:
sigma_1, sigma_2 = 0.1, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [21]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.16971483826637268
The normalized std of the hedge error is: 0.0557357557117939


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.3
$$

In [22]:
sigma_1, sigma_2 = 0.1, 0.3

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [23]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 1.369853138923645
The normalized std of the hedge error is: 0.06112033128738403


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.1
$$

In [24]:
sigma_1, sigma_2 = 0.3, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [25]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.81443190574646
The normalized std of the hedge error is: 0.06281157582998276


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.2 \quad \sigma_2 = 0.2 \quad \rho = 0.5
$$

In [26]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = 0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [27]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.46479177474975586
The normalized std of the hedge error is: 0.061586618423461914


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.0
$$

In [28]:
rho = 0.0

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [29]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.019558504223823547
The normalized std of the hedge error is: 0.059460531920194626


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = -0.2
$$

In [30]:
rho = -0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [31]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.05439833551645279
The normalized std of the hedge error is: 0.05264061689376831


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.2
$$

In [32]:
rho = 0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [33]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.014626985415816307
The normalized std of the hedge error is: 0.057423125952482224


### Different rebalancing periods

$$
N = 121
$$

In [34]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 121
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [35]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.35516199469566345
The normalized std of the hedge error is: 0.07791232317686081


$$
N = 52
$$

In [36]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 52
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [37]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.47219622135162354
The normalized std of the hedge error is: 0.11919223517179489


In [38]:
res_df = pd.DataFrame.from_dict(results)
res_df

,S1_0,S2_0,Volatility_1,Volatility_2,Maturity,Rebalancing_freq,Correlation,E_PL_T,Std_PL_T
0,100.0,100.0,0.2,0.2,1.0,252,-0.5,0.3005712,0.052476678
1,100.0,110.0,0.2,0.2,1.0,252,-0.5,0.3308047,0.037693582
2,100.0,120.0,0.2,0.2,1.0,252,-0.5,0.26628518,0.027673945
3,100.0,80.0,0.2,0.2,1.0,252,-0.5,0.24087866,0.12205779
4,100.0,90.0,0.2,0.2,1.0,252,-0.5,0.27034986,0.08254069
5,100.0,100.0,0.3,0.3,1.0,252,-0.5,0.49791008,0.052342974
6,100.0,100.0,0.1,0.1,1.0,252,-0.5,0.16971484,0.055735756
7,100.0,100.0,0.1,0.3,1.0,252,-0.5,1.3698531,0.06112033
8,100.0,100.0,0.3,0.1,1.0,252,-0.5,-0.8144319,0.062811576
9,100.0,100.0,0.2,0.2,1.0,252,0.5,-0.46479177,0.06158662


In [39]:
res_df.to_csv('./results/margrabe_delta_zero.csv')

## Hedging experiment with low transaction costs

In [40]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.01

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [41]:
results = {
    'S1_0': [],
    'S2_0': [],
    'Volatility_1': [],
    'Volatility_2': [],
    'Maturity': [],
    'Rebalancing_freq': [],
    'Correlation': [],
    'E_PL_T': [],
    'Std_PL_T': []
}

In [42]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.21832206845283508
The normalized std of the hedge error is: 0.054358888417482376


### Different starting stock prices

$$
S_1(0) = 100 \quad S_2(0) = 110
$$

In [43]:
s1_0 = 100
s2_0 = 110

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [44]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.25730669498443604
The normalized std of the hedge error is: 0.041085630655288696


$$
S_1(0) = 100 \quad S_2(0) = 120
$$

In [45]:
s1_0 = 100
s2_0 = 120

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [46]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.23251210153102875
The normalized std of the hedge error is: 0.028402747586369514


$$
S_1(0) = 100 \quad S_2(0) = 80
$$

In [47]:
s1_0 = 100
s2_0 = 80

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [48]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.16526374220848083
The normalized std of the hedge error is: 0.11387898772954941


$$
S_1(0) = 100 \quad S_2(0) = 90
$$

In [49]:
s1_0 = 100
s2_0 = 90

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [50]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.1635812222957611
The normalized std of the hedge error is: 0.07839024066925049


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.3
$$

In [51]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.3, 0.3
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [52]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.5149011611938477
The normalized std of the hedge error is: 0.0555887334048748


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1
$$

In [53]:
sigma_1, sigma_2 = 0.1, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [54]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.13425327837467194
The normalized std of the hedge error is: 0.055662740021944046


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.3
$$

In [55]:
sigma_1, sigma_2 = 0.1, 0.3

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [56]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 1.3853529691696167
The normalized std of the hedge error is: 0.06086169183254242


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.1
$$

In [57]:
sigma_1, sigma_2 = 0.3, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [58]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.8013604879379272
The normalized std of the hedge error is: 0.06317941844463348


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.2 \quad \sigma_2 = 0.2 \quad \rho = 0.5
$$

In [59]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = 0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [60]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.4971129596233368
The normalized std of the hedge error is: 0.06387461721897125


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.0
$$

In [61]:
rho = 0.0

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [62]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.03898519650101662
The normalized std of the hedge error is: 0.05534921586513519


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = -0.2
$$

In [63]:
rho = -0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [64]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.04373753443360329
The normalized std of the hedge error is: 0.05525830388069153


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.2
$$

In [65]:
rho = 0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [66]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.0025193197652697563
The normalized std of the hedge error is: 0.055398471653461456


### Different rebalancing periods

$$
N = 121
$$

In [67]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 121
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [68]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.3598994314670563
The normalized std of the hedge error is: 0.07980332523584366


$$
N = 52
$$

In [69]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 52
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [70]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.43382665514945984
The normalized std of the hedge error is: 0.12226552516222


In [71]:
res_df = pd.DataFrame.from_dict(results)
res_df

,S1_0,S2_0,Volatility_1,Volatility_2,Maturity,Rebalancing_freq,Correlation,E_PL_T,Std_PL_T
0,100.0,100.0,0.2,0.2,1.0,252,-0.5,0.21832207,0.05435889
1,100.0,110.0,0.2,0.2,1.0,252,-0.5,0.2573067,0.04108563
2,100.0,120.0,0.2,0.2,1.0,252,-0.5,0.2325121,0.028402748
3,100.0,80.0,0.2,0.2,1.0,252,-0.5,0.16526374,0.11387899
4,100.0,90.0,0.2,0.2,1.0,252,-0.5,0.16358122,0.07839024
5,100.0,100.0,0.3,0.3,1.0,252,-0.5,0.51490116,0.055588733
6,100.0,100.0,0.1,0.1,1.0,252,-0.5,0.13425328,0.05566274
7,100.0,100.0,0.1,0.3,1.0,252,-0.5,1.385353,0.06086169
8,100.0,100.0,0.3,0.1,1.0,252,-0.5,-0.8013605,0.06317942
9,100.0,100.0,0.2,0.2,1.0,252,0.5,-0.49711296,0.06387462


In [72]:
res_df.to_csv('./results/margrabe_delta_low.csv')

## Hedging experiment with high transaction costs

In [73]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.05

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [74]:
results = {
    'S1_0': [],
    'S2_0': [],
    'Volatility_1': [],
    'Volatility_2': [],
    'Maturity': [],
    'Rebalancing_freq': [],
    'Correlation': [],
    'E_PL_T': [],
    'Std_PL_T': []
}

In [75]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.2091313898563385
The normalized std of the hedge error is: 0.053352516144514084


### Different starting stock prices

$$
S_1(0) = 100 \quad S_2(0) = 110
$$

In [76]:
s1_0 = 100
s2_0 = 110

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [77]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.20903727412223816
The normalized std of the hedge error is: 0.03815138339996338


$$
S_1(0) = 100 \quad S_2(0) = 120
$$

In [78]:
s1_0 = 100
s2_0 = 120

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [79]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.17509210109710693
The normalized std of the hedge error is: 0.02705230936408043


$$
S_1(0) = 100 \quad S_2(0) = 80
$$

In [80]:
s1_0 = 100
s2_0 = 80

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [81]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.20510469377040863
The normalized std of the hedge error is: 0.11972278356552124


$$
S_1(0) = 100 \quad S_2(0) = 90
$$

In [82]:
s1_0 = 100
s2_0 = 90

s0 = jnp.array([s1_0, s2_0])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [83]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.17351306974887848
The normalized std of the hedge error is: 0.07768113911151886


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.3
$$

In [84]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.3, 0.3
rho = -0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [85]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.47807326912879944
The normalized std of the hedge error is: 0.05599630996584892


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1
$$

In [86]:
sigma_1, sigma_2 = 0.1, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [87]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.15839439630508423
The normalized std of the hedge error is: 0.054243817925453186


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.3
$$

In [88]:
sigma_1, sigma_2 = 0.1, 0.3

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [89]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 1.3821407556533813
The normalized std of the hedge error is: 0.062321737408638


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.3 \quad \sigma_2 = 0.1
$$

In [90]:
sigma_1, sigma_2 = 0.3, 0.1

vols = jnp.array([sigma_1, sigma_2])

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [91]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.8245880007743835
The normalized std of the hedge error is: 0.06309407949447632


### Different volatilites

$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.2 \quad \sigma_2 = 0.2 \quad \rho = 0.5
$$

In [92]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = 0.5
expiration = 1.0
N = 252
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [93]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.5131537914276123
The normalized std of the hedge error is: 0.06574606150388718


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.0
$$

In [94]:
rho = 0.0

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [95]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.058293040841817856
The normalized std of the hedge error is: 0.054197780787944794


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = -0.2
$$

In [96]:
rho = -0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [97]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.058347299695014954
The normalized std of the hedge error is: 0.052655063569545746


$$
S_1(0) = 100 \quad S_2(0) = 100 \quad \sigma_1 = 0.1 \quad \sigma_2 = 0.1 \quad \rho = 0.2
$$

In [98]:
rho = 0.2

correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [99]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: -0.01221650093793869
The normalized std of the hedge error is: 0.05760524421930313


### Different rebalancing periods

$$
N = 121
$$

In [100]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 121
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [101]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.37242016196250916
The normalized std of the hedge error is: 0.07581684738397598


$$
N = 52
$$

In [102]:
s1_0, s2_0 = 100, 100
mu_1, mu_2 = 0.0, 0.0
sigma_1, sigma_2 = 0.2, 0.2
rho = -0.5
expiration = 1.0
N = 52
dt = expiration / N
n_simulation = 1000
ticksize = 0.00

s0 = jnp.array([s1_0, s2_0])
means = jnp.array([0.0, 0.0])
vols = jnp.array([sigma_1, sigma_2])
correlation = jnp.array([[1, rho], [rho, 1]], dtype=jnp.float32)

gbm = MultiGeometricBrownianMotion(s0, means, vols, correlation, dtype=jnp.float32)
stock_paths = gbm.sample_paths(expiration, N, n_simulation)

In [103]:
results = update_results(stock_paths, expiration, vols, correlation, N, ticksize)

The mean hedge error is: 0.4295654594898224
The normalized std of the hedge error is: 0.12191732972860336


In [104]:
res_df = pd.DataFrame.from_dict(results)
res_df

,S1_0,S2_0,Volatility_1,Volatility_2,Maturity,Rebalancing_freq,Correlation,E_PL_T,Std_PL_T
0,100.0,100.0,0.2,0.2,1.0,252,-0.5,-0.20913139,0.053352516
1,100.0,110.0,0.2,0.2,1.0,252,-0.5,-0.20903727,0.038151383
2,100.0,120.0,0.2,0.2,1.0,252,-0.5,-0.1750921,0.02705231
3,100.0,80.0,0.2,0.2,1.0,252,-0.5,-0.2051047,0.11972278
4,100.0,90.0,0.2,0.2,1.0,252,-0.5,-0.17351307,0.07768114
5,100.0,100.0,0.3,0.3,1.0,252,-0.5,0.47807327,0.05599631
6,100.0,100.0,0.1,0.1,1.0,252,-0.5,0.1583944,0.054243818
7,100.0,100.0,0.1,0.3,1.0,252,-0.5,1.3821408,0.062321737
8,100.0,100.0,0.3,0.1,1.0,252,-0.5,-0.824588,0.06309408
9,100.0,100.0,0.2,0.2,1.0,252,0.5,-0.5131538,0.06574606


In [105]:
res_df.to_csv('./results/margrabe_delta_high.csv')